In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow==2.4.0
!pip install keras==2.4.0
!pip install kymatio
!pip install tqdm
!pip install iterative-stratification
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 394.7 MB 18 kB/s 
     |████████████████████████████████| 2.9 MB 53.9 MB/s 
     |████████████████████████████████| 14.8 MB 53.8 MB/s 
     |████████████████████████████████| 462 kB 72.9 MB/s 
     |████████████████████████████████| 132 kB 70.0 MB/s 
     |████████████████████████████████| 3.8 MB 58.0 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68727 sha256=29a272639a88cb9b05660d1e0d1edf0ae342daed92e9543536c93d75b951b967
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: numpy
    Found existing installation: n

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 170 kB 34.2 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 1.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 8.7 MB/s 


In [ ]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import pickle
import sys
sys.path.append("drive/MyDrive/Scattering_Novo/src")
from DataHandler import DataHandler
from ModelHandler import ModelHandler
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
 
configs = {
    "N_GRIDS": 5, 
    "SIGNAL_BASE_LENGTH": 12800, 
    "N_CLASS": 26, 
    "USE_NO_LOAD": False, 
    "USE_HAND_AUGMENTATION": True,
    "MARGIN_RATIO": 0.15, 
    "DATASET_PATH": "drive/MyDrive/YOLO_NILM/Synthetic_Full_iHall.hdf5",
    "TRAIN_SIZE": 0.9,
    "FOLDER_PATH": "drive/MyDrive/Scattering_Novo/tmp/DIFDUAL/tests/ANDH100_4/", 
    "FOLDER_DATA_PATH": "drive/MyDrive/Scattering_Novo/tmp/Without_Detection_With_HAND/NDH100/", 
    "N_EPOCHS_TRAINING": 5000,
    "PERCENTUAL": [1],
    "INITIAL_EPOCH": 0,
    "TOTAL_MAX_EPOCHS": 5000,
    "SNRdb": None # Nível de ruído em db
}

def freeze(model, task_name='classification'):
    for layer in model.layers:
        if task_name in layer.name:
            layer.trainable = True
        else:
            layer.trainable = False

    for layer in model.layers:
        print(layer.name, layer.trainable)

    return model

def calculating_class_weights(y_true):
    '''
        Source: https://stackoverflow.com/questions/48485870/multi-label-classification-with-class-weights-in-keras
    '''
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight(class_weight='balanced', classes=[0.,1.], y=y_true[:, i])
    return weights


def reduce_dataset(X_all,ydet_all,ytype_all,yclass_all,percentual):
    import numpy as np
    max_index = int(percentual*X_all.shape[0])
    np.random.seed(100)
    index = np.random.randint(max_index,size=(max_index-1))
    X_all = X_all[index]
    ydet_all = ydet_all[index]
    ytype_all = ytype_all[index]
    yclass_all = yclass_all[index]

    return X_all,ydet_all,ytype_all,yclass_all

ngrids = configs["N_GRIDS"]
signalBaseLength = configs["SIGNAL_BASE_LENGTH"]
trainSize = configs["TRAIN_SIZE"]
folderDataPath = configs["FOLDER_DATA_PATH"]
 
dataHandler = DataHandler(configs)

if not os.path.isfile(folderDataPath + "data.p"):
    print("Sorted data not found, creating new file...")
    x, ydet, yclass, ytype, ygroup = dataHandler.loadData(hand_augmentation=configs["USE_HAND_AUGMENTATION"], SNR=configs["SNRdb"])
    print("Data loaded")

    data_mskf = MultilabelStratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    strat_classes = np.max(yclass, axis=1)
    train_index, test_index = next(data_mskf.split(x, strat_classes))

    y_train = {
        "detection": ydet[train_index], 
        "type": ytype[train_index], 
        "classification": yclass[train_index], 
        "group": ygroup[train_index]
    }
    
    y_test = {
        "detection": ydet[test_index], 
        "type": ytype[test_index], 
        "classification": yclass[test_index], 
        "group": ygroup[test_index]
    }
    
    dict_data = {
        "x_train": x[train_index], 
        "x_test": x[test_index], 
        "y_train": y_train, 
        "y_test": y_test
    }

    print("Data sorted")

    try:
        os.mkdir(folderDataPath)
    except:
        pass

    pickle.dump(dict_data, open(folderDataPath + "data.p", "wb"))
    print("Data stored")
else:
    dict_data = pickle.load(open(folderDataPath + "data.p", "rb"))



modelHandler = ModelHandler(configs)
 
X_all = dict_data["x_train"]
ydet_all = dict_data["y_train"]["detection"]
ytype_all = dict_data["y_train"]["type"]
yclass_all = dict_data["y_train"]["classification"]

if configs["PERCENTUAL"][0]!=1:
        X_all,ydet_all,ytype_all,yclass_all = reduce_dataset(X_all,ydet_all,ytype_all,yclass_all,configs["PERCENTUAL"][0])

print(X_all.shape)
print(dict_data["x_test"].shape)


Using TensorFlow backend.


(18723, 16640, 1)
(2089, 16640, 1)


In [ ]:
configs["PERCENTUAL"][0]

1

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, GlobalAveragePooling1D, Flatten, MaxPool1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from kymatio.keras import Scattering1D

def buildBaseScattering(input_shape):
    '''
        Source: https://github.com/kymatio/kymatio/blob/master/examples/1d/classif_keras.py
    '''
    log_eps = 1e-6
    
    input = Input(shape=(input_shape,))
    x = Scattering1D(10, 10, max_order=1)(input) # Changed J from 8 to 10 -> Results in a flatten with 544 parameters (the original with convolutions has 520)
    

    unmapped_len = int(0.15 * (x.shape[2] / 1.3))
    grid_len = int((x.shape[2] - 2 * unmapped_len) / 5)

    print(f"X: {x.shape[2]}, Unmapped: {unmapped_len}, Grid: {grid_len}")

    left = Lambda(lambda x: x[..., :, : unmapped_len], name='left')(x)
    center = Lambda(lambda x: x[..., :, unmapped_len : x.shape[2] - unmapped_len], name='center')(x)
    right = Lambda(lambda x: x[..., :, x.shape[2] - unmapped_len :], name='right')(x)

    g1 = Lambda(lambda x: x[..., :, :grid_len], name='g1')(center)
    g2 = Lambda(lambda x: x[..., :, grid_len:2*grid_len], name='g2')(center)
    g3 = Lambda(lambda x: x[..., :, 2*grid_len:3*grid_len], name='g3')(center)
    g4 = Lambda(lambda x: x[..., :, 3*grid_len:4*grid_len], name='g4')(center)
    g5 = Lambda(lambda x: x[..., :, 4*grid_len:], name='g5')(center)


    leftav = tf.keras.backend.max(left, axis=2)
    g1av   = tf.keras.backend.max(g1, axis=2)
    g2av   = tf.keras.backend.max(g2, axis=2)
    g3av   = tf.keras.backend.max(g3, axis=2)
    g4av   = tf.keras.backend.max(g4, axis=2)
    g5av   = tf.keras.backend.max(g5, axis=2)
    rightav = tf.keras.backend.max(right, axis=2)
    
    

    x_type = tf.concat([(g1av-leftav), (g2av-leftav), (g3av-g1av), (g4av-g2av), (g5av-g3av), (rightav-g4av), (rightav-g5av) ], axis=1)


    x_class = tf.concat([leftav, g1av, g2av, g3av, g4av, g5av, rightav], axis=1)
  
    x_type = Flatten()(x_type)
    x_class = Flatten()(x_class)


    model = Model(inputs = input, outputs=[x_type, x_class])

    return model


tf.keras.backend.clear_session()

scattering_extract = buildBaseScattering(X_all.shape[1])
scattering_extract.summary()

X: 17, Unmapped: 1, Grid: 3
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 16640)]      0                                            
__________________________________________________________________________________________________
scattering1d (Scattering1D)     (None, 86, 17)       0           input_1[0][0]                    
__________________________________________________________________________________________________
center (Lambda)                 (None, 86, 15)       0           scattering1d[0][0]               
__________________________________________________________________________________________________
g1 (Lambda)                     (None, 86, 3)        0           center[0][0]                     
__________________________________________________________________

In [ ]:
scattering_extract.save(configs["FOLDER_PATH"] + 'scattering_model.h5')

## Treinamento

In [ ]:
fold = 0
mskf = MultilabelStratifiedKFold(n_splits=10, shuffle=True, random_state=42)
strat_classes = np.max(yclass_all, axis=1)
print(strat_classes.shape)

for train_index, validation_index in mskf.split(X_all, strat_classes):
    fold += 1

    if fold != 6:
        continue

    print(f"---------- FOLD {fold} -------------")

    scaler = MaxAbsScaler()
    scaler.fit(np.squeeze(X_all[train_index], axis=2))
    x_train = np.expand_dims(scaler.transform(np.squeeze(X_all[train_index], axis=2)), axis=2)
    x_validation = np.expand_dims(scaler.transform(np.squeeze(X_all[validation_index], axis=2)), axis=2)
    
    
    x_train_type, x_train_class = scattering_extract.predict(x_train)
    x_validation_type, x_validation_class = scattering_extract.predict(x_validation)

    # Normalizing

    transformer = MaxAbsScaler().fit(x_train_type)
    x_train_type = transformer.transform(x_train_type)
    
    transformer = MaxAbsScaler().fit(x_train_class)
    x_train_class = transformer.transform(x_train_class)

    transformer = MaxAbsScaler().fit(x_validation_type)
    x_validation_type = transformer.transform(x_validation_type)

    transformer = MaxAbsScaler().fit(x_validation_class)
    x_validation_class = transformer.transform(x_validation_class)

    print("Each grid has size " + str(x_train[1].shape))

    y_train, y_validation = {}, {}
    y_train["detection"] = ydet_all[train_index]
    y_validation["detection"] = ydet_all[validation_index]
    y_train["type"] = ytype_all[train_index]
    y_validation["type"] = ytype_all[validation_index]
    y_train["classification"] = yclass_all[train_index]
    y_validation["classification"] = yclass_all[validation_index]

    yclass_weights = calculating_class_weights(np.max(y_train["classification"], axis=1))

    print(yclass_weights)
    
    folderPath = configs["FOLDER_PATH"] + str(fold) + "/"
    try:
        os.mkdir(folderPath)
    except:
        pass

    np.save(folderPath + "train_index.npy", train_index)
    np.save(folderPath + "validation_index.npy", validation_index)
    

    tensorboard_callback = TensorBoard(log_dir='./' + configs["FOLDER_PATH"] + '/logs')


    if configs["INITIAL_EPOCH"] > 0:
        model = ModelHandler.loadModel(folderPath + 'model_{0}.h5'.format(configs["INITIAL_EPOCH"]))
    else:
        model = modelHandler.buildScatteringOutput3(input_shape=x_train_type.shape[1]) 
 
    model.summary()
 
    fileEpoch = configs["INITIAL_EPOCH"]
    while fileEpoch < configs["TOTAL_MAX_EPOCHS"]:
        fileEpoch += configs["N_EPOCHS_TRAINING"]      

        if not os.path.isfile(folderPath + 'model_without_detection.h5'):
            for subtask in ['type', 'classification']:
                print(f"FOLD {fold}: Training {subtask}")
                
                freeze(model, task_name=subtask)
                model.compile(optimizer = Adam(), \
                            
                              loss = ["categorical_crossentropy", "binary_crossentropy"], \
                              metrics=[['categorical_accuracy'], ['binary_accuracy']])
                
                early_stopping_callback = EarlyStopping(monitor=f"val_{subtask}_loss", patience=50, verbose=True, restore_best_weights=True)
                
                hist_opt = model.fit(x=[x_train_type, x_train_class], y=[y_train["type"], y_train["classification"]], \
                                    validation_data=([x_validation_type, x_validation_class], [y_validation["type"], y_validation["classification"]]), \
                                    epochs=configs["N_EPOCHS_TRAINING"], verbose=2, callbacks=[early_stopping_callback, tensorboard_callback], batch_size=32)
                
                   
            model.save(folderPath + 'model_without_detection.h5')
  
    del model, y_validation, y_train, x_validation, x_train

(18723, 26)
---------- FOLD 6 -------------
Each grid has size (16640, 1)
[[ 0.5498953   5.5104918 ]
 [ 0.55083246  5.41811734]
 [ 0.50302287 83.2029703 ]
 [ 0.5604949   4.63257993]
 [ 0.57585829  3.79561879]
 [ 0.50389758 64.64230769]
 [ 0.51867053 13.89008264]
 [ 0.61245536  2.72310434]
 [ 0.54799478  5.70889946]
 [ 0.50544328 46.4281768 ]
 [ 0.51159747 22.05643045]
 [ 0.55652318  4.92296426]
 [ 0.66992188  1.97126437]
 [ 0.57573993  3.80076888]
 [ 0.50696791 36.37878788]
 [ 0.57696533  3.74821588]
 [ 0.56965157  4.0892944 ]
 [ 0.51739318 14.87345133]
 [ 0.52666708  9.87485311]
 [ 0.51768003 14.6402439 ]
 [ 0.52746046  9.604     ]
 [ 0.53072502  8.63669065]
 [ 0.53477791  7.6884721 ]
 [ 0.52397431 10.92782835]
 [ 0.53693055  7.26946367]
 [ 0.53335237  7.99571836]]
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 

## Treinamento antigo (Usado com as CNNs)